In [1]:
! pip install torch torchvision diffusers transformers accelerate safetensors pillow

In [2]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image, ImageDraw

In [3]:
# Choose a model from Hugging Face
model_id = "runwayml/stable-diffusion-v1-5"  # You can change this to another model

In [4]:
device = (
    torch.device("cuda") if torch.cuda.is_available()
    else torch.device("mps") if torch.backends.mps.is_available()
    else torch.device("cpu")
)
print(f"Using device: {device}")

Using device: mps


In [5]:
# Load the pipeline
pipe = StableDiffusionInpaintPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
def make_circle_mask(width, height, radius=None):
    # Create a black image (keep area)
    mask = Image.new("L", (width, height), 0)
    draw = ImageDraw.Draw(mask)

    # Default radius = half width
    if radius is None:
        radius = min(width, height) // 3

    # Center of the image
    cx, cy = width // 2, height // 2

    # Draw white circle (inpaint area)
    draw.ellipse(
        (cx - radius, cy - radius, cx + radius, cy + radius),
        fill=255
    )

    # Convert to RGB because diffusers expects RGB
    return mask.convert("RGB")

In [7]:
width, height = 512, 512
mask_image = make_circle_mask(width, height, 100)
mask_image.save("mask.png")

In [8]:
# Load your input image
# base_image: the image you want to modify
# mask_image: white = inpaint area, black = keep area
base_image = Image.open("img2img_result.png").convert("RGB")
mask_image = Image.open("mask.png").convert("RGB")   # must be RGB for the pipeline

In [9]:
# Optional but recommended: ensure both are the same size and divisible by 8
W, H = base_image.size
new_W, new_H = W - W % 8, H - H % 8
base_image = base_image.resize((new_W, new_H))
mask_image = mask_image.resize((new_W, new_H))

In [10]:
# Text conditioning
prompt = "A fantasy dragon replacing the masked region, highly detailed"

In [11]:
# Run the inpainting pipeline
result = pipe(
    prompt=prompt,
    image=base_image,
    mask_image=mask_image,
    num_inference_steps=50,
    guidance_scale=7.5,
    strength=0.8,   # how much to transform the masked region
).images[0]

  0%|          | 0/40 [00:00<?, ?it/s]

In [12]:
# Save the generated image
result.save("inpaint_result.png")
print("Saved inpaint_result.png")

Saved inpaint_result.png
